In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from dotenv import load_dotenv
from datetime import datetime
import pandas as pd
import os
import time

load_dotenv()


# Check whether the specified path exists or not
isExist = os.path.exists("output")

if not isExist:
  
  # Create a new directory because it does not exist 
  os.makedirs("output")

# Check whether the specified path exists or not
isExist = os.path.exists("output/html")

if not isExist:
  
  # Create a new directory because it does not exist 
  os.makedirs("output/html")

# inicializa o Sellenium
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

browser = webdriver.Chrome('/usr/bin/chromedriver', options=chrome_options)

# Get Login Page
browser.get("https://sistemaweb.hubert.com.br/")
html = browser.page_source
time.sleep(2)

lgoin = browser.find_element(by=By.NAME, value="Login")
lgoin.send_keys(os.getenv("USER"))

password = browser.find_element(by=By.NAME, value="Senha")
password.send_keys(os.getenv("PASS"))

# Submit Login
browser.find_element(by=By.CLASS_NAME, value="btn-lg").click()


time.sleep(2)

print(f"{os.getenv('FROM')}-01")
print(datetime.now().strftime("%Y-%m-%d"))
# Get Range of dates
months = pd.date_range(start=f"{os.getenv('FROM')}-01", end=datetime.now(
).strftime("%Y-%m-%d"), freq='MS').strftime("%m-%Y").tolist()
print(months)

for month in months:
    # Efetua o download dos arquivos HTML
    browser.get(
        f"https://sistemaweb.hubert.com.br/Financeiro/PrestacaoContas/{month}")
    time.sleep(2)

    html = browser.page_source
    print(f"get {month}.html")
    # Salva os Arquivos
    f = open(f"output/html/{month}.html", "w")
    f.write(html)
    f.close()

browser.close()
# Clean files without report


/tmp/ipykernel_651/1650866833.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('/usr/bin/chromedriver', options=chrome_options)


2020-04-01
2022-04-13
['04-2020', '05-2020', '06-2020', '07-2020', '08-2020', '09-2020', '10-2020', '11-2020', '12-2020', '01-2021', '02-2021', '03-2021', '04-2021', '05-2021', '06-2021', '07-2021', '08-2021', '09-2021', '10-2021', '11-2021', '12-2021', '01-2022', '02-2022', '03-2022', '04-2022']
get 04-2020.html
get 05-2020.html
get 06-2020.html
get 07-2020.html
get 08-2020.html
get 09-2020.html
get 10-2020.html
get 11-2020.html
get 12-2020.html
get 01-2021.html
get 02-2021.html
get 03-2021.html
get 04-2021.html
get 05-2021.html
get 06-2021.html
get 07-2021.html
get 08-2021.html
get 09-2021.html


KeyboardInterrupt: 

In [2]:

import pandas as pd
from datetime import datetime
import os

from bs4 import BeautifulSoup

# Get Range of dates
months = pd.date_range(start=f"{os.getenv('FROM')}-01", end=datetime.now(
).strftime("%Y-%m-%d"), freq='MS').strftime("%m-%Y").tolist()
print(months)

# Limpa os arquivos sem dados
for month in list(months):
    try:
        f = open(f"output/html/{month}.html", "r")
        bs = BeautifulSoup(f.read(), 'html.parser')
        if (not bs.find("div", "clsReportSheet")):
            months.remove(month)
            os.remove(f"output/html/{month}.html")
    except IOError:
        print("File not accessible")
        months.remove(month)
    finally:
        f.close()
print(months)


['04-2020', '05-2020', '06-2020', '07-2020', '08-2020', '09-2020', '10-2020', '11-2020', '12-2020', '01-2021', '02-2021', '03-2021', '04-2021', '05-2021', '06-2021', '07-2021', '08-2021', '09-2021', '10-2021', '11-2021', '12-2021', '01-2022', '02-2022', '03-2022', '04-2022']
['04-2020', '05-2020', '06-2020', '07-2020', '08-2020', '09-2020', '10-2020', '11-2020', '12-2020', '01-2021', '02-2021', '03-2021', '04-2021', '05-2021', '06-2021', '07-2021', '08-2021', '09-2021', '10-2021', '11-2021', '12-2021', '01-2022', '02-2022']


In [6]:
import re
import numpy as np

# Efetua a leitura da tabela Receita
def readReceita(rec):
    # busca a tabela
    table = rec.findNext("table")
    tableStr = table.prettify()
    tableStr = tableStr.replace("colspan=\"5\"", "")
    # transforma HTML em Dataframe
    df = pd.read_html(tableStr, thousands='.', decimal=',')
    df = df[0]
    #Renomeia e Insere colunas
    df.insert(0, "Conta", account, allow_duplicates=True)
    df.insert(0, "RecDesp", recdesp, allow_duplicates=True)
    df.insert(2, "Mes", date, allow_duplicates=True)
    if(len(df.index) > 1):
        df.drop(df.tail(1).index, inplace=True)
    df.rename(columns={'Unnamed: 0': 'Nome', 'Inadimplência Atual':'InadimplenciaAtual'}, inplace=True)
    return df


# Efetua a leitura da tabela Despesa
def readDespesa(rec):
    # busca a tabela
    table = rec.findNext("table")
    tableStr = table.prettify()
    tableStr = tableStr.replace("colspan=\"5\"", "")
    # transforma HTML em Dataframe
    df = pd.read_html(tableStr, thousands='.', decimal=',')
    df = df[0]
    #Renomeia e Insere colunas
    df.insert(0, "Conta", account, allow_duplicates=True)
    df.insert(0, "RecDesp", recdesp, allow_duplicates=True)
    df.insert(2, "Mes", date, allow_duplicates=True)
    df.rename(columns={0: 'DespesaId', 2: 'Descricao',
              3: 'Valor', 4: 'Total', 5: 'Porcentagem'}, inplace=True)
    df.drop(columns=[1], inplace=True)
    # Preenche a categoria da despesa, visto que ela se encontra em  uma liha separada
    categoria = ''
    categorias = []
    indexToRemove = []
    for index, row in df.iterrows():
        if type(row['DespesaId']) == str and not row['DespesaId'].isnumeric():
            categoria = row['DespesaId']
            indexToRemove.append(index)
        row['Categoria'] = categoria
        categorias.append(categoria)
    # Add coluna Tipo
    df.insert(3, "Tipo", categorias, allow_duplicates=True)
    # remove linha do Tipo
    df.drop(indexToRemove, axis=0, inplace=True)
    # Remove ultima linha total
    df.drop(df.tail(1).index, inplace=True)
    # df.to_csv(f'temp-{account}.csv')
    return df

# Efetua a leitura da tabela Resumo
def readResumo(rec):
    # busca a tabela
    table = rec.findNext("table")
    tableStr = table.prettify()
    tableStr = tableStr.replace("colspan=\"5\"", "")
    # transforma HTML em Dataframe
    df = pd.read_html(tableStr, thousands='.', decimal=',')
    df = df[0]
    #Renomeia e Insere colunas
    df.insert(0, "Conta", account, allow_duplicates=True)
    df.insert(0, "RecDesp", recdesp, allow_duplicates=True)
    df.insert(2, "Mes", date, allow_duplicates=True)
    df.rename(columns={0: 'Nome', 1: 'Valor'}, inplace=True)
    df = df[df["Nome"].str.contains("SALDO") == False]
    # print(df)
    return df
    # Efetua a leitura da tabela Resumo
def readSaldoResumo(rec):
    # busca a tabela
    table = rec.findNext("table")
    tableStr = table.prettify()
    tableStr = tableStr.replace("colspan=\"5\"", "")
    # transforma HTML em Dataframe
    df = pd.read_html(tableStr, thousands='.', decimal=',')
    df = df[0]
    #Renomeia e Insere colunas
    df.insert(0, "Conta", account, allow_duplicates=True)
    df.insert(0, "RecDesp", recdesp, allow_duplicates=True)
    df.insert(2, "Mes", date, allow_duplicates=True)
    df.rename(columns={0: 'Nome', 1: 'Valor'}, inplace=True)
    df = df[df["Nome"].str.contains("SALDO") == True]
    df = df[df["Nome"].str.contains("ANTERIOR") == False]
    # print(df)
    return df

# Efetua a leitura da tabela Final
def readFinal(rec):    
    table = rec.findNext("table")
    tableStr = table.prettify()
    tableStr = tableStr.replace("colspan=\"5\"", "")
    df = pd.read_html(tableStr, thousands='.', decimal=',')
    df = df[0]
    df.insert(0, "Mes", date, allow_duplicates=True)
    df.rename(columns={0: 'Nome', 1: 'Valor'}, inplace=True)
    
    df = df[df["Nome"].str.contains("SALDO ACUMULADO") == True]
    # print(df)
    return df

receita = []
despesa = []
resumo = []
saldoresumo = []
conta = []

# Inicializa o Arquivo Excel
with pd.ExcelWriter('output.xlsx') as writer:
    
    for month in months:

        date = datetime.strptime(month,'%m-%Y').date()
        f = open(f"output/html/{month}.html", "r")
        bs = BeautifulSoup(f.read(), 'html.parser')
        reportSheet = bs.find("div", "clsReportSheet")
        headers = reportSheet.find_all("div", "clsAccountHeader")
        accounts = []
        # Busca os Headers
        for header in headers:
            # print(header.find("span").text)
            headerValue = header.find("span").text
            # Busca o Número da Conta
            account = re.search("^[0-9]*", headerValue).group()
            if account != '':
                # Busca a descrição Receita/Despesa
                recdesp = re.findall("[\w]+", headerValue)[1]

                accounts.append({'Id': account, "Nome": headerValue.replace(f'{account} - {recdesp}', '')})

                if(recdesp == "RECEITAS"):
                    receita.append(readReceita(header))
                if (recdesp == "DESPESAS"):
                    despesa.append(readDespesa(header))
                if (recdesp == "RESUMO"):
                    resumo.append(readResumo(header))
                    saldoresumo.append(readSaldoResumo(header))
            if "RESUMO FINAL CONTA CORRENTE" == headerValue:
                conta.append(readFinal(header))

    # Concatena Dataframe com os dados extraídos                
    dfReceita = pd.concat(receita)
    dfDespesa = pd.concat(despesa)
    dfResumo = pd.concat(resumo)
    dfSaldoResumo = pd.concat(saldoresumo)
    dfConta = pd.concat(conta)

    #Grava os DataFrames no Excel em diferentes abas
    dfReceita.to_excel(writer, sheet_name="Receita", index=False)
    dfDespesa.to_excel(writer, sheet_name="Despesa", index=False)
    dfResumo.to_excel(writer, sheet_name="Resumo", index=False)
    dfConta.to_excel(writer, sheet_name="Saldo", index=False)

    dfReceita.to_csv('output/receita.csv', index=False)
    dfDespesa.to_csv('output/despesa.csv', index=False)
    dfResumo.to_csv('output/resumo.csv', index=False)
    dfConta.to_csv('output/saldo.csv', index=False)
    dfSaldoResumo.to_csv('output/saldoresumo.csv', index=False)

    #Limpa a lista de Contas
    seen_acounts = set()
    new_list = []
    for obj in accounts:
        if obj['Id'] not in seen_acounts:
            new_list.append(obj)
            seen_acounts.add(obj['Id'])
    dfAccounts = pd.DataFrame(new_list)
    dfAccounts.to_excel(writer, sheet_name="Contas", index=False)
    dfAccounts.to_csv('output/contas.csv', index=False)
    # print(dfAccounts)



IndexError: At least one sheet must be visible